In [1]:
from pathlib import Path
import pandas as pd
import seaborn as sns
from src.data.make_dataset import load_data

In [2]:
dataset_path = "../data/raw/AirlineQuotesData.xlsx" 
raw_data = load_data(dataset_path, "Flight Duplicate Quotes")

# Deduplicate

The analyst has come up with the following criteria to determine if quotes should be considered duplicates of each other:
1. The quotes are made by the same user (“UserID”)
2. The flight dates are within 14 days of each other (“FlightDate”)


In [3]:
raw_data.shape

(2152, 6)

In [4]:
raw_data.sort_values("QuoteCreationDateTime", inplace=True) # sort by quote creation date

# FYI Quotes being made after Flight

There are some quotes created after the flight has taken place.

In [5]:
raw_data[raw_data.QuoteCreationDateTime.dt.date > raw_data.FlightDate]

,QuoteID,UserID,QuoteCreationDateTime,FlightDate,Price,Sale_Flag
1173,QUOTE_1174,USER_148,2018-05-04 15:14:00,2018-05-02,285.12,0
1488,QUOTE_1489,USER_17,2018-07-04 13:49:00,2018-06-24,263.70,0
1833,QUOTE_1834,USER_54,2018-09-17 14:22:00,2018-09-12,236.70,0


# Get Number of Days from prior search

In [6]:
raw_data["NoDaysFlightDatePriorSearch"] = (raw_data.groupby("UserID")["FlightDate"].diff() / pd.Timedelta(days=1)).fillna(0.)

raw_data["NoDaysFlightDatePriorSearch"] = raw_data["NoDaysFlightDatePriorSearch"].abs()

In [7]:
raw_data[raw_data.UserID =='USER_158']

,QuoteID,UserID,QuoteCreationDateTime,FlightDate,Price,Sale_Flag,NoDaysFlightDatePriorSearch
2146,QUOTE_2147,USER_158,2018-12-29 11:51:00,2019-01-07,638.85,0,0.0
2147,QUOTE_2148,USER_158,2018-12-29 11:55:00,2019-01-07,644.56,0,0.0
2148,QUOTE_2149,USER_158,2018-12-29 11:58:00,2019-01-14,527.70,0,7.0
2149,QUOTE_2150,USER_158,2018-12-31 11:51:00,2019-01-28,514.12,1,14.0
2150,QUOTE_2151,USER_158,2019-02-01 11:51:00,2019-03-05,737.41,0,36.0
2151,QUOTE_2152,USER_158,2019-02-01 11:55:00,2019-03-05,730.23,0,0.0


In [8]:
def deduplicate(group):
    "Deduplicate criteria given by analyst."
    filtered = group[
        # keep quotes that are greater than 14 days of prior search
        (group.NoDaysFlightDatePriorSearch > 14) |
        
        # keep all sale rows
        (group.Sale_Flag == 1)

        ]
    
    return filtered

In [9]:
raw_data[raw_data.UserID=="USER_158"].groupby("UserID").apply(lambda x: deduplicate(x))

QuoteID    UserID QuoteCreationDateTime FlightDate   Price   
UserID                                                                         
USER_158 2149  QUOTE_2150  USER_158   2018-12-31 11:51:00 2019-01-28  514.12  \
         2150  QUOTE_2151  USER_158   2019-02-01 11:51:00 2019-03-05  737.41   

               Sale_Flag  NoDaysFlightDatePriorSearch  
UserID                                                 
USER_158 2149          1                         14.0  
         2150          0                         36.0

After doing this, I still didn't feel this solution was robust enough. If multiple trips where being planned at the same time, the above would not be able to handle this. 

Because of this, I then sorted the flight date and grouped them by there flight date search delta - see python scripts.